![](https://raw.githubusercontent.com/mission-young/Pic/master/img/20190630115256.png)

尽管$\beta$叠加修正依赖于$\beta$能谱,但本文的工作证明了对Q值不敏感，Q值表示的是到末态而不是基态。

Me为电子质量，Q是母核到子核基态的能量差，Mr为子核基态质量，MI为质心系的总能量. 单位是MeV。

In [1]:
%jsroot on
static const Double_t Me = 0.511e-3;  
static const Double_t Q = 18.58e-3;	     
static const Double_t Mr = 20493e-3;  
static const Double_t MI = Mr+Q+Me;

Q值设定为Al22基态到Mg22基态的Q值。Mr为Mg22基态质量（GeV）。
![](https://i.loli.net/2019/07/02/5d1b13f529b5025157.png)

![](https://i.loli.net/2019/07/02/5d1b1488ec62b94122.png)
上图为[TGenPhaseSpace](https://root.cern/doc/v618/classTGenPhaseSpace.html#ab1ffe458ad5dd053d4ebab308553595a)类的介绍，适用于质心系下多体反应计算。

![](https://i.loli.net/2019/07/02/5d1b17f104caa50216.png)
![](https://i.loli.net/2019/07/02/5d1b1836104ff66038.png)
![](https://i.loli.net/2019/07/02/5d1b17cc6b84294106.png)

In [2]:
void beta(){
    if(gRandom) delete gRandom;
    gRandom=new TRandom(0);
    TLorentzVector* p[3]={0,0,0};
    TGenPhaseSpace* rg=new TGenPhaseSpace();
    TLorentzVector* p0=new TLorentzVector(0,0,0,MI);
    Double_t masses[3]={0,Me,Mr};
    rg->SetDecay((*p0),3,masses,"Fermi");

    TCanvas *c=new TCanvas();
    TH1D *h=new TH1D("h","h",200,0,20e-3);
    for(int i=0;i<1000000;i++){
        double w1=rg->Generate();
        for(int j=0;j<3;j++) p[j]=rg->GetDecay(j);
        double e=p[1]->E();
        double w2=w1*(p[1]->E()*p[0]->E());
        h->Fill(e,w2);
    }
    h->Draw();
    c->Draw();
}
beta();

上述为从[root论坛](https://root-forum.cern.ch/t/weighting-in-event-generator/2532)上找到的示例代码经简化做出的$\beta$能谱。下面为原程序。

In [3]:
/**
   maximum momentum 
 */
double p3Max(double m, double m1, double m2, double m3) { 
  double a = (m1+m2-m3);
  double b = (m1+m2+m3);
  double pmax = sqrt( (m*m -a*a)*(m*m - b*b) )/2/m;
  return pmax; 
}


In [4]:


/** 
 this is the correct function from excact phase space calculation
*/
double pDist( double * x, double * par  ) {
  
  double m = MI;
  double m1 = Mr;
  double m2 = 0;
  double m3 = Me;
  double p3 = x[0];
  double e3 = sqrt(p3*p3 + m3*m3);
  double pmax = p3Max(m,m1,m2,m3);
  if (p3 > pmax) return 0; 

  double a = ( m*m - 2*m*e3 + m3*m3 - (m1-m2)*(m1-m2) );
  double b = ( m*m - 2*m*e3 + m3*m3 - (m1+m2)*(m1+m2) );
  double c = 2* (m*m + m3*m3 - 2*m*e3);
  double fval = p3*p3/2./e3*sqrt(a*b)/c;
  return par[0]*fval;
}


In [5]:

double eDist( double * x, double * par  ) {
  
  double m3 = Me;
  double e3 = x[0];
  if ( e3 < m3) return 0;
  double p3 = sqrt(e3*e3 - m3*m3);
  double xp[1]; xp[0] = p3; 
  double pfval = pDist(xp,par);
  double fval = pfval*e3/p3;
  return fval;
}


In [6]:

// famous beta distribution
double eDist2(double * x,double * par)  {
  
  double m3 = Me;
  double e3 = x[0];
  if ( e3 < m3 || e3>Q+m3) return 0;
  double p3 = sqrt(e3*e3 - m3*m3);
  double fval=par[0]*(Q+m3-e3)*(Q+m3-e3)*p3*e3;
  return fval;
}


In [7]:


void doit(UInt_t NrOfEvents = 1000000)   //***Number of events can be changed here or when you want to run by "doit(...)"
{
  if (gRandom) delete gRandom;
  gRandom = new TRandom3(0); 
  TLorentzVector* p[3] = {0,0,0};
  TGenPhaseSpace* rg = new TGenPhaseSpace();
  TLorentzVector* p0 = new TLorentzVector(0,0,0,MI); 
  Double_t masses[3] = {0,Me,Mr};
  rg->SetDecay((*p0), 3, masses,"Fermi");
 
  TCanvas* betacan=new TCanvas("bcan","event generator for decay",300,10,900,800);  //***defining CANVAS
   
  TH1D* E1 = new TH1D("E1","E_1",200,0,20e-3);
  TH1D* E1_2 = new TH1D("E1_2","E_1 (2)",2000,0,20e-3);
  TH1D* E2 = new TH1D("E2","E_2",200,0,20e-3);
  TH1D* P1 = new TH1D("P1","P_1",200,0,20e-3);
  TH1D* P2 = new TH1D("P2","P_2",200,0,20e-3);

    
  betacan->Divide(2,2);   
  gStyle->SetPalette(1);          
    
  Double_t weight1;
  Double_t weight2;
  Double_t weightC1;
  Double_t weightC2;
  Double_t Eb,Pb;
     
  for (UInt_t i = 0; i<NrOfEvents ; i++)   
  {
      weight1 = rg->Generate();     
      
     for(UInt_t j = 0; j<3; j++) p[j] = rg->GetDecay(j);     
      
    Eb=p[1]->E();
    Pb=p[1]->P();

    weight2=weight1*(p[1]->E()*p[0]->E());
            
    E1->Fill(Eb,weight1);
    E2->Fill(Eb,weight2);	
    E1_2->Fill(Eb,weight1);
    P1->Fill(Pb,weight1);
    P2->Fill(Pb,weight2);
    }
 
  betacan->cd(1);
  E1->Draw();
  betacan->cd(2); 
  E2->Draw();
  betacan->cd(3);
  P1->Draw();
  betacan->cd(4); 
  P2->Draw();
 

  // fit functions 
  gStyle->SetOptFit(1111);          
  TF1 * pFunc = new TF1("pDist",pDist,0.,1.2,1);    
  TF1 * eFunc = new TF1("EDist",eDist,0.,1.3,1);
  TF1 * eFunc2 = new TF1("EDist2",eDist2,0.,1.3,1);
  pFunc->SetParameter(0,1.0);
  eFunc->SetParameter(0,100.0);
  betacan->cd(1);
  E1->Fit(eFunc2);
  betacan->cd(2);
  E2->Fit(eFunc2);
  betacan->cd(3);
  P1->Fit(pFunc);
  betacan->cd(4);
  P2->Fit(pFunc);
  betacan->Draw();
}
doit();

 FCN=63430.8 FROM MIGRAD    STATUS=CONVERGED      17 CALLS          18 TOTAL
                     EDM=1.15669e-20    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.23408e+06   1.38769e+03   1.65958e+02   1.09605e-13
 FCN=217.345 FROM MIGRAD    STATUS=CONVERGED      19 CALLS          20 TOTAL
                     EDM=9.69833e-23    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.60534e+01   1.03954e-01   6.02532e-02   1.33974e-10
 FCN=180.282 FROM MIGRAD    STATUS=CONVERGED      14 CALLS          15 TOTAL
                     EDM=3.43432e-12    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE

分别生成Q值为1、2、3、4、5、6、7、8、9、10、11、12、13、14、15、16、17、18、19的$\beta$能谱，并存到文件中，便于模拟调用。

In [8]:
void getbeta(){
    TFile *f=new TFile("beta.root","recreate");
    if(gRandom) delete gRandom;
    gRandom=new TRandom(0);
    TLorentzVector* p[3]={0,0,0};
    TGenPhaseSpace* rg=new TGenPhaseSpace();
    
    double Mre=(0.511+20493)*1e-3;
    char name[32];
    for(int i=1;i<20;i++){
        TLorentzVector* p0=new TLorentzVector(0,0,0,Mre+i*1e-3);
        Double_t masses[3]={0,0.511e-3,20493e-3};
        rg->SetDecay((*p0),3,masses,"Fermi");
        sprintf(name,"Q%02dMeV",i);
        TH1D h(name,name,200,0,20e-3);
        for(int i=0;i<1000000;i++){
            double w1=rg->Generate();
            for(int j=0;j<3;j++) p[j]=rg->GetDecay(j);
            double e=p[1]->E();
            double w2=w1*(p[1]->E()*p[0]->E());
            h.Fill(e,w2);
        }
        h.Write();
    }
    f->Close();
}
getbeta();

In [9]:
TFile *f=new TFile("beta.root");
f->ls();

TFile**		beta.root	
 TFile*		beta.root	
  KEY: TH1D	Q01MeV;1	Q01MeV
  KEY: TH1D	Q02MeV;1	Q02MeV
  KEY: TH1D	Q03MeV;1	Q03MeV
  KEY: TH1D	Q04MeV;1	Q04MeV
  KEY: TH1D	Q05MeV;1	Q05MeV
  KEY: TH1D	Q06MeV;1	Q06MeV
  KEY: TH1D	Q07MeV;1	Q07MeV
  KEY: TH1D	Q08MeV;1	Q08MeV
  KEY: TH1D	Q09MeV;1	Q09MeV
  KEY: TH1D	Q10MeV;1	Q10MeV
  KEY: TH1D	Q11MeV;1	Q11MeV
  KEY: TH1D	Q12MeV;1	Q12MeV
  KEY: TH1D	Q13MeV;1	Q13MeV
  KEY: TH1D	Q14MeV;1	Q14MeV
  KEY: TH1D	Q15MeV;1	Q15MeV
  KEY: TH1D	Q16MeV;1	Q16MeV
  KEY: TH1D	Q17MeV;1	Q17MeV
  KEY: TH1D	Q18MeV;1	Q18MeV
  KEY: TH1D	Q19MeV;1	Q19MeV


In [10]:
TCanvas c;
char name[32];
TH1D *hq[19];
for(int i=1;i<20;i++){
    sprintf(name,"Q%02dMeV",i);
    hq[i-1]=(TH1D*)f->Get(name);
}
hq[18]->Draw();
for(int i=1;i<19;i++){
    hq[i-1]->SetLineColor(i*2);
    hq[i-1]->Draw("same");
}
c.Draw();

In [11]:
for(int i=0;i<19;i++){
    hq[i]->Scale(1000000/hq[i]->Integral(0,299));
}

In [12]:
c.Clear();
hq[0]->Draw();
for(int i=1;i<19;i++)
hq[i]->Draw("same");
c.BuildLegend();
c.Draw();

![](https://i.loli.net/2019/07/02/5d1b3ca0bddb938233.png)

In [13]:
void getbeta2(){
    TFile *f=new TFile("beta2.root","recreate");
    if(gRandom) delete gRandom;
    gRandom=new TRandom(0);
    TLorentzVector* p[3]={0,0,0};
    TGenPhaseSpace* rg=new TGenPhaseSpace();
    
    double Mre=(0.511+62377)*1e-3;
    char name[32];
    for(int i=1;i<20;i++){
        TLorentzVector* p0=new TLorentzVector(0,0,0,Mre+i*1e-3);
        Double_t masses[3]={0,0.511e-3,62377e-3};
        rg->SetDecay((*p0),3,masses,"Fermi");
        sprintf(name,"Q%02dMeV",i);
        TH1D h(name,name,200,0,20e-3);
        for(int i=0;i<1000000;i++){
            double w1=rg->Generate();
            for(int j=0;j<3;j++) p[j]=rg->GetDecay(j);
            double e=p[1]->E();
            double w2=w1*(p[1]->E()*p[0]->E());
            h.Fill(e-511e-6,w2);
        }
        h.Write();
    
}
    f->Close();
}
getbeta2();

In [14]:
c.Clear();
TFile *f2=new TFile("beta2.root");
TH1D *hq2[19];
for(int i=1;i<20;i++){
    sprintf(name,"Q%02dMeV",i);
    hq2[i-1]=(TH1D*)f2->Get(name);
}
hq2[18]->Draw();
for(int i=1;i<19;i++){
    hq2[i-1]->SetLineColor(i*2);
    hq2[i-1]->Draw("same");
}
c.Draw();

In [15]:
for(int i=0;i<19;i++){
    hq2[i]->Scale(1000000/hq2[i]->Integral(0,299));
}

In [16]:
c.Clear();
hq2[4]->SetLineColor(1);
hq2[4]->Draw();
hq2[6]->Draw("same");
hq2[7]->Draw("same");
hq2[8]->Draw("same");
hq2[9]->Draw("same");
hq2[10]->Draw("same");
hq2[12]->Draw("same");
c.BuildLegend();
c.Draw();

文献中给出的图
<img src="https://i.loli.net/2019/07/02/5d1b3d7bc897160420.png" width="40%" height="25%" />

与文献相比，总体趋势相同，但差异在于最可几能量的位置。